In [1]:
# pip install midi
from midiutil.MidiFile import MIDIFile
import os


def string_parse(in_str):
    duration = in_str[1]
    note = in_str[0]
    midi_value = 79 - note 
    return duration, midi_value

def populate_midi(notes, tempo):
    path = "output.midi"
    mf = MIDIFile(1)     # only 1 track
    track = 0   # the only track
    time = 0    # start at the beginning
    channel = 0
    volume = 100
    tempo = int(tempo)
    mf.addTrackName(track, time, "Sample Track")
    mf.addTempo(track, time, tempo)
    for note_time, pitches in enumerate(notes):
        for pitch in pitches:
            duration, note_value = string_parse(pitch)
            print(note_time)
            mf.addNote(track, channel, note_value, note_time, duration, volume)
    
    with open(path, 'wb') as f: 
        mf.writeFile(f) 
    
    #descriptor = os.open(path=path, 
    #                     flags=(
    #                         os.O_WRONLY 
    #                         | os.O_CREAT
    #                         | os.O_TRUNC
    #                     ),
    #                     mode=0o777)

    #with open(descriptor, 'wb') as outf:
    #    mf.writeFile(outf)
    print(mf)
    return "output.midi"


In [5]:
import torch

# pip install miditok=2.1.5 VERY IMPORTANT
from miditok import REMI

from transformers import GPT2LMHeadModel
from miditoolkit import MidiFile

class MidiGenerator():
    def __init__(self):
        self.tokenizer = REMI(params = "GPT2Piano\\Tokenizer\\tokenizer.conf")
        self.model = GPT2LMHeadModel.from_pretrained("Qilex/GPT2Piano")
        self.device = torch.device(0)
        self.model.to(self.device)

    def condition_on_existing(self, midi_file):
        mid = MidiFile(midi_file, clip=True)
        # print(mid)
        tokens = self.tokenizer(mid)
        prompt = torch.tensor(tokens, dtype=torch.int64).to(self.device)
        continuation = self.model.generate(prompt, max_new_tokens=512,
                                           num_beams=1,
                                           do_sample=True,
                                           temperature=1.1,
                                           top_k=75,
                                           top_p=0.9,
                                           pad_token_id=self.tokenizer['PAD_None'])
        midi_result = self.tokenizer(continuation.cpu())
        midi_result.dump('continuation.midi')
        return "continuation.midi"

In [6]:
generator = MidiGenerator()


C:\Users\alexm\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
midi = generator.condition_on_existing("output.midi")


In [22]:
notes = [[[24, '1'], [36, '8']], [[20, '1']], [[12, '1']], [[20, '1']], [[0, '2'], [24, '1']], [[20, '1']], [[0, '2'], [12, '1']], [[20, '1']], [[1, '3'], [25, '1'], [29, '8']], [[22, '1']], [[13, '1']], [[3, '1'], [22, '1']], [[1, '2'], [25, '1']], [[22, '1']], [[1, '1'], [13, '1']], [[0, '1'], [22, '1']], [[3, '11'], [27, '8']], [[24, '1']], [[15, '1']], [[24, '1']], [], [[24, '1']], [[15, '1']], [[24, '1']], [[27, '1'], [31, '8']], [[19, '1']], [[15, '1']], [[19, '1']], [[3, '2'], [27, '1']], [[19, '1']], [[1, '2'], [15, '1']], [[19, '1']]]
tempo = 200
path = "output.midi"
mf = MIDIFile(1)     # only 1 track
track = 0   # the only track
time = 0    # start at the beginning
channel = 0
volume = 100
tempo = int(tempo)
mf.addTrackName(track, time, "Sample Track")
mf.addTempo(track, time, tempo)
for note_time, pitches in enumerate(notes):
    for pitch in pitches:
        duration, note_value = string_parse(pitch)
        print(f'Pitch is {note_value} beat is {note_time} and length is {duration}')
        mf.addNote(track, channel, note_value, note_time, duration, volume)

with open("tmp.mid", "wb") as output_file:
    mf.writeFile(output_file)

Pitch is 55 beat is 0 and length is 1
Pitch is 43 beat is 0 and length is 8
Pitch is 59 beat is 1 and length is 1
Pitch is 67 beat is 2 and length is 1
Pitch is 59 beat is 3 and length is 1
Pitch is 79 beat is 4 and length is 2
Pitch is 55 beat is 4 and length is 1
Pitch is 59 beat is 5 and length is 1
Pitch is 79 beat is 6 and length is 2
Pitch is 67 beat is 6 and length is 1
Pitch is 59 beat is 7 and length is 1
Pitch is 78 beat is 8 and length is 3
Pitch is 54 beat is 8 and length is 1
Pitch is 50 beat is 8 and length is 8
Pitch is 57 beat is 9 and length is 1
Pitch is 66 beat is 10 and length is 1
Pitch is 76 beat is 11 and length is 1
Pitch is 57 beat is 11 and length is 1
Pitch is 78 beat is 12 and length is 2
Pitch is 54 beat is 12 and length is 1
Pitch is 57 beat is 13 and length is 1
Pitch is 78 beat is 14 and length is 1
Pitch is 66 beat is 14 and length is 1
Pitch is 79 beat is 15 and length is 1
Pitch is 57 beat is 15 and length is 1
Pitch is 76 beat is 16 and length is 11


In [23]:
from miditoolkit import MidiFile
mid = MidiFile('tmp.mid', clip=True)


In [25]:
help(mid)

Help on MidiFile in module miditoolkit.midi.parser object:

class MidiFile(builtins.object)
 |  MidiFile(filename: Union[pathlib.Path, str, NoneType] = None, file=None, ticks_per_beat: int = 480, clip: bool = False, charset: str = 'latin1')
 |  
 |  Methods defined here:
 |  
 |  __eq__(self, other)
 |      Return self==value.
 |  
 |  __init__(self, filename: Union[pathlib.Path, str, NoneType] = None, file=None, ticks_per_beat: int = 480, clip: bool = False, charset: str = 'latin1')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  dump(self, filename: Union[pathlib.Path, str, NoneType] = None, file=None, segment: Optional[Tuple[int, int]] = None, shift: bool = True, instrument_idx: Optional[int] = None, charset: str = 'latin1')
 |  
 |  get_tick_to_time_mapping(self) -> numpy.ndarray
 |  
 |  ----------------------------------------------------------------

In [ ]:
string =[[['G3', '1'], ['G2', '8']], [['B3', '1']], [['G4', '1']], [['B3', '1']], [['G5', '2'], ['G3', '1']], [['B3', '1']], [['G5', '2'], ['G4', '1']], [['B3', '1']], [['F#5', '3'], ['F#3', '1'], ['D3', '8']], [['A3', '1']], [['F#4', '1']], [['E5', '1'], ['A3', '1']], [['F#5', '2'], ['F#3', '1']], [['A3', '1']], [['F#5', '1'], ['F#4', '1']], [['G5', '1'], ['A3', '1']], [['E5', '11'], ['E3', '8']], [['G3', '1']], [['E4', '1']], [['G3', '1']], [], [['G3', '1']], [['E4', '1']], [['G3', '1']], [['E3', '1'], ['C3', '8']], [['C4', '1']], [['E4', '1']], [['C4', '1']], [['E5', '2'], ['E3', '1']], [['C4', '1']], [['F#5', '2'], ['E4', '1']], [['C4', '1']]]

In [16]:
print(string)

[[['G3', '1'], ['G2', '8']], [['B3', '1']], [['G4', '1']], [['B3', '1']], [['G5', '2'], ['G3', '1']], [['B3', '1']], [['G5', '2'], ['G4', '1']], [['B3', '1']], [['F#5', '3'], ['F#3', '1'], ['D3', '8']], [['A3', '1']], [['F#4', '1']], [['E5', '1'], ['A3', '1']], [['F#5', '2'], ['F#3', '1']], [['A3', '1']], [['F#5', '1'], ['F#4', '1']], [['G5', '1'], ['A3', '1']], [['E5', '11'], ['E3', '8']], [['G3', '1']], [['E4', '1']], [['G3', '1']], [], [['G3', '1']], [['E4', '1']], [['G3', '1']], [['E3', '1'], ['C3', '8']], [['C4', '1']], [['E4', '1']], [['C4', '1']], [['E5', '2'], ['E3', '1']], [['C4', '1']], [['F#5', '2'], ['E4', '1']], [['C4', '1']]]


In [17]:
mf

NameError: name 'mf' is not defined

In [18]:
populate_midi(string, 180)

0
0
1
2
3
4
4
5
6
6
7
8
8
8
9
10
11
11
12
12
13
14
14
15
15
16
16
17
18
19
21
22
23
24
24
25
26
27
28
28
29
30
30
31


'output.midi'

In [9]:
from midiutil import MIDIFile

degrees  = [60, 62, 64, 65, 67, 69, 71, 72] # MIDI note number
track    = 0
channel  = 0
time     = 0   # In beats
duration = 1   # In beats
tempo    = 60  # In BPM
volume   = 100 # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1) # One track, defaults to format 1 (tempo track
                     # automatically created)
MyMIDI.addTempo(track,time, tempo)

for pitch in degrees:
    MyMIDI.addNote(track, channel, pitch, time, duration, volume)
    time = time + 1
    print(time)
    MyMIDI.addNote(track, channel, 65, 0, 3, volume)
    

with open("major-scale.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)

1
2
3
4
5
6
7
8
